# Sampling from a multivariate normal distribution
---

We look to implement a sampling method to gather samples from a multivariate normal distribution, using random samples from a uniform distribution, and we use JAX library to implement the sampling the method. We have been successful in extracting samples from a multivariate normal distribution to a `p-value` of as low as `0.0006`.<br>
The sampling method is described in detail below.

### **Parameters for the multivariate normal distribution**
We have the following parameters to generate samples :
1. Dimesion of the vector sampled from a multivariate mormal distribution, $d$ 
2. Number of sample vectors of $d$ dimesions, drawn from a multivariate normal distribution, taken as $N$
3. Mean vector for the distributed data, $\mu$
4. Covariance Matrix, $\Sigma$<br><br>
Any multivariate normal distibution will random vectors of different dimensions, that is why it is neccessary to mention that the any vector $X$ drawn from a multivariate normal distibution will have a certain dimension $d$
$$
X \sim N_d(\mu,\Sigma) = (x_1,x_2,...,x_d)^{T}
$$<br> 
A larger sample of data $N$ captures more finer points in a distribution and tends to be more accurate, but is similarly associated with a significant time and more easily exposes the minute flaw in a sampling method.<p>
The mean vector,  $μ$  is essentially the center of the distribution that result as the random samples from bivariate normal distribution. It's components  $μ_x$  and  $μ_y$  decribe the position of that center.<p>
A covariance matrix  $Σ$ , by definition, is symmetric and positive definite, which means  $a^⊤Σa>0$  for all  $a∈R^d$ . A necessary and sufficient condition for  $Σ$  to be positive definite is that all of its eigenvalues are positive.<p>

### **Our Sampling Method**
We have successfully developed a sampling method `mvn_samples()`, whereby we try to undertake the following steps : 
1. We first use `jax.numpy.linalgs.cholesky()` on our covariance matrix $K$ to get a Cholesky decomposition matrix $L$.
2. We then use `jax.random.uniform()` to first generate two numbers from a uniform distribution and then we use the Box Muller Transform to map them to two numbers in a normal distribution.
3. We generate a sample matrix $z$ with iteration on Step 2, which then undergoes a dot product, by using `np.dot()`, with our Cholesky decomposition matrix $L$ and added with our mean vector $\mu$, we get a matrix $x$ of order $d \times N$ which contains $N$ samples from a multivariate normal distribution of dimension $d$.

### **Box Muller Transform**
The Box Muller Transform allows us to sample from a pair of normally distributed variables using a source of only uniformly distributed variables. The transform is actually pretty simple to compute. We first sample two uniformly distributed random variables $U_1$ and $U_2$ using `jax.random.uniform()`. These two can then be mapped to two normally distributed variables $X$ and $Y$ as follows:
$$
X = R cos{Θ}
\\Y = R sin{Θ}
$$
where $R = \sqrt{2ln(U_1)}$ and $Θ = 2\pi U_2$. This hence helps us with sampling for even dimensions. But for odd dimensions, an extra step of discarding an edge value of the sample is enough to help us generalise this algorithm for any dimension $d$.

### **Error Calculations on the sampled data**
We use `scipy.stats.kstest` to run a Kolmogorov-Smirnov Test on our sampled data. We, by using this test, pit our sampling method against a random sampling method over a multivariate normally distributed data. This test returns a `p-value` which is significant in our hypothesis testing.<p>
**Interpreting P-Value** : A lower p-value is associated with a low error to occur when null hypothesis is rejected. In its very basic sense, it is a quantitative measure of : *How likely is the data, given that the null hypothesis is true?*<br>
The null hypothesis for our experiment is : "The sampled data is insignificant"
We have gathered an enough low p-value so as to successfully reject the null hypothesis.

---
### **Code** 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import jax
import jax.numpy as jnp
import random
import scipy.stats as stats
import math

In [3]:
# dimension
d = 10

# no. of samples
N = 10000

# mean vector
mu = jnp.array([0]*(d*N)).reshape(d,N)

# set cov matrix (transform)
K = jnp.identity(d)

In [4]:
# Cholesky Decomposition on K
L = jnp.linalg.cholesky(K)

In [5]:
def mvn_samples(dim,num_samples):
    i = 1
    j = 1

    z_o = []
    z = []

    while i <= num_samples:
        while j <= dim:
            key1 = jax.random.PRNGKey(random.randint(1, 1000))
            key2 = jax.random.PRNGKey(random.randint(1, 1000))

            u1 = jax.random.uniform(key1)
            u2 = jax.random.uniform(key2)

            n1 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
            n2 = math.sqrt(-2 * math.log(u1)) * math.sin(2 * math.pi * u2)

            z_o.append(n1)
            z_o.append(n2)
            j += 2
        if d%2 == 1: z_o.pop()
        j = 1
        i += 1
    z = np.array(z_o).reshape(d,N)
    return z

In [6]:
# Using our sampling method to generate N samples of d dimensions
z = mvn_samples(d,N)

In [7]:
x = mu + np.dot(L, z)
x

DeviceArray([[-0.4937135 ,  0.62204444,  0.8375828 , ...,  1.398562  ,
               0.5243517 ,  0.78688836],
             [ 0.8740825 ,  0.20310648, -0.6787484 , ...,  0.9197775 ,
              -0.36174822,  1.4511871 ],
             [ 0.7331297 ,  0.9659406 , -0.6709099 , ...,  0.5670589 ,
               0.79626155,  0.69782096],
             ...,
             [-0.41330576, -0.12295498, -1.1500207 , ...,  0.32302696,
               0.06485996,  1.1162292 ],
             [-0.9599803 ,  1.097428  ,  0.1392937 , ...,  0.13469744,
              -0.43227312, -1.1445631 ],
             [-0.653419  , -0.48580903, -0.8741085 , ...,  0.4594269 ,
              -2.1363518 ,  1.0201639 ]], dtype=float32)

In [12]:
# Error Calculations on the sampled data
for i in range(d):
    test_stat, pvalue = stats.kstest(x[i], 'norm', args=(0, 1), N=N*d)
    print("sample_N[%d](mu,K) vs. N(0, 1): KS=%.4f with p-value = %.4f." % (i,test_stat, pvalue))

sample_N[0](mu,K) vs. N(0, 1): KS=0.0107 with p-value = 0.1999.
sample_N[1](mu,K) vs. N(0, 1): KS=0.0120 with p-value = 0.1120.
sample_N[2](mu,K) vs. N(0, 1): KS=0.0097 with p-value = 0.3043.
sample_N[3](mu,K) vs. N(0, 1): KS=0.0190 with p-value = 0.0015.
sample_N[4](mu,K) vs. N(0, 1): KS=0.0184 with p-value = 0.0024.
sample_N[5](mu,K) vs. N(0, 1): KS=0.0135 with p-value = 0.0516.
sample_N[6](mu,K) vs. N(0, 1): KS=0.0166 with p-value = 0.0082.
sample_N[7](mu,K) vs. N(0, 1): KS=0.0133 with p-value = 0.0583.
sample_N[8](mu,K) vs. N(0, 1): KS=0.0106 with p-value = 0.2095.
sample_N[9](mu,K) vs. N(0, 1): KS=0.0201 with p-value = 0.0006.


### **References**
1. [Multivariate Normal Distribution](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjWysnjgor3AhUhCqYKHRypCRIQFnoECB0QAQ&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FMultivariate_normal_distribution&usg=AOvVaw3C61xV1Xhz133Z12_TArIW)

2. [Generation of Normally Distributed Numbers](https://www.projectrhea.org/rhea/index.php/Generation_of_N-dimensional_normally_distributed_random_numbers_from_two_categories_with_different_priors)

3. [Box Muller Transform](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform)

4. [Cholesky Decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition)